In [1]:
################ Letter-Recognition #################
# Disable info messages from the tesnorflow
import os,math
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
################## Import the libraries #####################
import numpy as np
import pandas as pd
import keras
from keras.models import Model, Sequential
from keras.utils import to_categorical
from keras.layers import Input, Dense, Dropout, Flatten, Conv2D,MaxPooling2D
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,StandardScaler,MinMaxScaler,scale
from sklearn.metrics import accuracy_score
from scipy.ndimage import zoom

In [2]:
############### Load the dataset ###################
path = 'covtype.csv'
classes = 7
data=pd.read_csv(path, header=None)
X=data.iloc[:,:54]
Y=data.iloc[:,-1]
X = MinMaxScaler().fit_transform(X)
Y = LabelEncoder().fit_transform(Y)
Y = to_categorical(Y,classes)
print(X.shape,Y.shape)

(581012, 54) (581012, 7)


In [3]:
X_train,X_test, Y_train,Y_test = train_test_split(X,Y,test_size=0.483659)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)
y_train = np.argmax(Y_train,axis=-1)

(300000, 54) (300000, 7)
(281012, 54) (281012, 7)


In [4]:
from tab2img.converter import Tab2Img
model = Tab2Img()
train = model.fit_transform(X_train, y_train)
test = model.transform(X_test)
print(train.shape,test.shape)

(300000, 8, 8) (281012, 8, 8)


In [5]:
from PIL import Image
x_train = []
x_test = []
for i in range(len(train)):
    img = Image.fromarray(train[i])
    imgs = img.resize(size=(32, 32))
    imgs = np.array(imgs)
    x_train.append(np.repeat(imgs[:, :, np.newaxis], 3, axis=2))
x_train = np.array(x_train,dtype='float32')
for i in range(len(test)):
    img = Image.fromarray(test[i])
    imgs = img.resize(size=(32, 32))
    imgs = np.array(imgs)
    x_test.append(np.repeat(imgs[:, :, np.newaxis], 3, axis=2))
x_test = np.array(x_test,dtype='float32')
print(x_train.shape,x_test.shape)

(300000, 32, 32, 3) (281012, 32, 32, 3)


In [7]:
############### Feature-extraction ###############
# pretrained model for FE
pretrained_model = keras.applications.DenseNet121(input_shape=(32,32,3), weights=None, include_top=False)
x = Flatten()(pretrained_model.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
prediction = Dense(classes, activation='softmax')(x)
model = Model(inputs=pretrained_model.input, outputs=prediction)
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)
from keras import callbacks 
earlystopping = callbacks.EarlyStopping(monitor ="val_loss",  
                                        mode ="min", patience = 5,  
                                        restore_best_weights = True) 
  
model.fit(x_train,Y_train,batch_size=256, epochs=20,verbose=1,validation_data=(x_test,Y_test), callbacks =[earlystopping]) 

Epoch 1/20
1172/1172 [==============================] - 105s 90ms/step - loss: 0.5851 - accuracy: 0.7538 - val_loss: 0.7062 - val_accuracy: 0.7115
Epoch 2/20
1172/1172 [==============================] - 99s 84ms/step - loss: 0.4451 - accuracy: 0.8136 - val_loss: 0.5258 - val_accuracy: 0.7832
Epoch 3/20
1172/1172 [==============================] - 99s 85ms/step - loss: 0.3767 - accuracy: 0.8439 - val_loss: 0.4304 - val_accuracy: 0.8283
Epoch 4/20
1172/1172 [==============================] - 99s 85ms/step - loss: 0.3293 - accuracy: 0.8647 - val_loss: 0.5450 - val_accuracy: 0.7984
Epoch 5/20
1172/1172 [==============================] - 99s 85ms/step - loss: 0.2933 - accuracy: 0.8794 - val_loss: 0.4816 - val_accuracy: 0.8206
Epoch 6/20
1172/1172 [==============================] - 99s 85ms/step - loss: 0.2655 - accuracy: 0.8916 - val_loss: 0.2958 - val_accuracy: 0.8766
Epoch 7/20
1172/1172 [==============================] - 100s 85ms/step - loss: 0.2412 - accuracy: 0.9020 - val_loss: 0.3177

In [8]:
score = model.evaluate(x_test,Y_test)
acc = model.evaluate(x_train,Y_train)
print('training accuracy: ',acc[1])
print('testing accuracy: ',score[1])

9375/9375 [==============================] - 84s 9ms/step - loss: 0.1241 - accuracy: 0.9491
training accuracy:  0.9490600228309631
testing accuracy:  0.9345152378082275
